In [2]:
import pandas as pd, numpy as np, os, glob
csv_path = "Student_performance_data _.csv"
if not os.path.exists(csv_path):
    matches = glob.glob("Student_performance_data*.csv")
    if not matches:
        raise FileNotFoundError("CSV not found. Update csv_path to the correct location.")
    csv_path = matches[0]
encodings = ["utf-8", "utf-8-sig", "latin-1"]
df, last_err = None, None
for enc in encodings:
    try:
        df = pd.read_csv(csv_path, encoding=enc)
        break
    except Exception as e:
        last_err = e
if df is None:
    raise last_err


def clean_col(c):
    c = str(c).strip().lower().replace("\n"," ")
    c = "".join(ch if ch.isalnum() else "_" for ch in c)
    c = "_".join(filter(None, c.split("_")))
    return c
df.columns = [clean_col(c) for c in df.columns]


empty_cols = [c for c in df.columns if df[c].isna().all()]
df.drop(columns=empty_cols, inplace=True)


for c in df.columns:
    if df[c].dtype == "object":
        num = pd.to_numeric(df[c].astype(str).str.replace(",",""), errors="coerce")
        if num.notna().mean() >= 0.8:
            df[c] = num
            continue
        dt = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
        if dt.notna().mean() >= 0.8:
            df[c] = dt
df = df.drop_duplicates().reset_index(drop=True)

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
dt_cols  = df.select_dtypes(include=["datetime64[ns]","datetime64[ns, UTC]"]).columns.tolist()
obj_cols = df.select_dtypes(include=["object"]).columns.tolist()

fill_values = {}
for c in num_cols:
    if df[c].isna().any():
        med = df[c].median()
        if np.isfinite(med):
            fill_values[c] = med
for c in dt_cols:
    if df[c].isna().any() and not df[c].dropna().empty:
        m = df[c].mode()
        if not m.empty: fill_values[c] = m.iloc[0]
for c in obj_cols:
    if df[c].isna().any():
        m = df[c].mode()
        fill_values[c] = (m.iloc[0] if not m.empty else "Unknown")
df = df.fillna(fill_values)
def qbands(s, labels):
    q = s.quantile([0.33, 0.66]).tolist()
    bins = [-np.inf, q[0], q[1], np.inf]
    return pd.cut(s, bins=bins, labels=labels, include_lowest=True)

if "gpa" in df.columns:
    gmin, gmax = df["gpa"].min(), df["gpa"].max()
    if gmax <= 5:
        bins  = [-np.inf, 2.0, 3.0, 3.5, np.inf]
        labs  = ["Low (<2.0)", "Medium (2.0–3.0)", "High (3.0–3.5)", "Very High (>=3.5)"]
        df["gpa_band"] = pd.cut(df["gpa"], bins=bins, labels=labs, include_lowest=True)
    else:
        df["gpa_band"] = qbands(df["gpa"], ["Low (Q1)","Medium (Q2)","High (Q3)","Very High (Q4)"])

if "studytimeweekly" in df.columns:
    df["study_time_band"] = qbands(df["studytimeweekly"], ["Low study time","Medium study time","High study time"])

if "absences" in df.columns:
    df["absence_band"] = qbands(df["absences"], ["Low absences","Moderate absences","High absences"])

activity_cols = [c for c in ["extracurricular","sports","music","volunteering"] if c in df.columns]
if activity_cols:
    df["activity_count"] = df[activity_cols].sum(axis=1, min_count=1)

if "tutoring" in df.columns:
    df["is_tutored"] = (df["tutoring"] > 0).astype(int)

if "parentalsupport" in df.columns:
    df["parental_support_band"] = qbands(df["parentalsupport"], ["Low support","Medium support","High support"])

# === 8) Example filtering & aggregation ===
# Filter: rows with positive study time
filtered = df[df.get("studytimeweekly", 0) > 0]
aggs = {}
if "gender" in df.columns and "gpa" in df.columns:
    aggs["gpa_by_gender"] = df.groupby("gender")["gpa"].mean().reset_index()
if "ethnicity" in df.columns and "gpa" in df.columns:
    aggs["gpa_by_ethnicity"] = df.groupby("ethnicity")["gpa"].mean().reset_index()

# === 9) Save outputs ===
base = os.path.splitext(os.path.basename(csv_path))[0]
clean_path     = f"{base}_cleaned.csv"
enriched_path  = f"{base}_cleaned_enriched.csv"
df.to_csv(enriched_path, index=False)
df.drop(columns=[c for c in ["gpa_band","study_time_band","absence_band","activity_count","is_tutored","parental_support_band"] if c in df.columns], errors="ignore").to_csv(clean_path, index=False)

print("Saved:", os.path.abspath(clean_path))
print("Saved:", os.path.abspath(enriched_path))
print(f"Rows: {len(df)}, Columns: {df.shape[1]}")


Saved: C:\Users\anjan\final project\Student_performance_data __cleaned.csv
Saved: C:\Users\anjan\final project\Student_performance_data __cleaned_enriched.csv
Rows: 2392, Columns: 21
